# 작업형2

## [가격 예측] 중고 자동차
- 자동차 가격을 예측해주세요!


- 예측할 값(y): price
- 평가: RMSE (Root Mean Squared Error)
- data: train.csv, test.csv
- 제출 형식: result.csv파일을 아래와 같은 형식(수치형)으로 제출
~~~
pred
11000
20500
19610
...
11995
~~~

### 답안 제출 참고
- pd.read_csv('result.csv') 로 제출 코드 확인

In [1]:
# 데이터 불러오기
import pandas as pd

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

## EDA

In [2]:
# 데이터 크기 확인
train.shape, test.shape

((3759, 9), (1617, 8))

In [3]:
# 샘플 확인 (시험환경에서는 display 대신 print 활용)
print(train.head(5))
print(test.head(5))

       model  year  price transmission  mileage fuelType  tax   mpg  \
0   EcoSport  2017  10298       Manual    25013   Petrol  150  53.3   
1      Focus  2016  10491       Manual    30970   Diesel    0  74.3   
2      S-MAX  2017  13498       Manual    60200   Diesel  150  56.5   
3     Fiesta  2018  10600       Manual    27380   Petrol  145  56.5   
4     Fiesta  2018  11000       Manual     7724   Petrol  145  65.7   

   engineSize  
0         1.0  
1         1.5  
2         2.0  
3         1.0  
4         1.0  
     model  year transmission  mileage fuelType  tax   mpg  engineSize
0    S-MAX  2016       Manual    23905   Diesel  125  56.5         2.0
1   Fiesta  2018       Manual    16895   Petrol  145  40.3         1.5
2     Kuga  2017       Manual    12109   Petrol  165  45.6         1.5
3     Kuga  2018    Semi-Auto    13940   Diesel  145  58.9         1.5
4   Fiesta  2013       Manual    31690   Petrol   30  54.3         1.2


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3759 entries, 0 to 3758
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         3759 non-null   object 
 1   year          3759 non-null   int64  
 2   price         3759 non-null   int64  
 3   transmission  3759 non-null   object 
 4   mileage       3759 non-null   int64  
 5   fuelType      3759 non-null   object 
 6   tax           3759 non-null   int64  
 7   mpg           3759 non-null   float64
 8   engineSize    3759 non-null   float64
dtypes: float64(2), int64(4), object(3)
memory usage: 264.4+ KB


In [5]:
# 기초 통계값 확인
train.describe()

,year,price,mileage,tax,mpg,engineSize
count,3759.000000,3759.000000,3759.000000,3759.000000,3759.000000,3759.000000
mean,2016.887736,12353.321362,23191.786379,114.444001,57.793083,1.352807
std,2.151811,4750.338051,19223.258713,61.713682,10.099221,0.437130
min,2002.000000,1095.000000,5.000000,0.000000,20.900000,0.000000
25%,2016.000000,9000.000000,9997.500000,75.000000,52.300000,1.000000
50%,2017.000000,11495.000000,18180.000000,145.000000,58.900000,1.200000
75%,2018.000000,15310.000000,31000.000000,145.000000,65.700000,1.500000
max,2060.000000,48000.000000,144000.000000,570.000000,201.800000,5.000000


In [7]:
test.describe()

,year,mileage,tax,mpg,engineSize
count,1617.000000,1617.000000,1617.000000,1617.000000,1617.000000
mean,2016.752628,23817.544836,113.089054,57.804082,1.362894
std,2.182152,20130.026364,63.584868,10.567025,0.436878
min,2005.000000,5.000000,0.000000,20.900000,0.000000
25%,2016.000000,10422.000000,30.000000,52.300000,1.000000
50%,2017.000000,18471.000000,145.000000,58.900000,1.200000
75%,2018.000000,31098.000000,145.000000,65.700000,1.600000
max,2020.000000,150890.000000,570.000000,201.800000,5.000000


In [8]:
train.describe(include = 'object')

,model,transmission,fuelType
count,3759,3759,3759
unique,19,3,3
top,Fiesta,Manual,Petrol
freq,1372,3234,2541


In [9]:
test.describe(include = 'object')

,model,transmission,fuelType
count,1617,1617,1617
unique,19,3,3
top,Fiesta,Manual,Petrol
freq,591,1385,1085


In [10]:
print(train['transmission'].value_counts())
print(test['transmission'].value_counts())


transmission
Manual       3234
Automatic     312
Semi-Auto     213
Name: count, dtype: int64
transmission
Manual       1385
Automatic     127
Semi-Auto     105
Name: count, dtype: int64


In [11]:
print(train.isnull().sum())
print(test.isnull().sum())

model           0
year            0
price           0
transmission    0
mileage         0
fuelType        0
tax             0
mpg             0
engineSize      0
dtype: int64
model           0
year            0
transmission    0
mileage         0
fuelType        0
tax             0
mpg             0
engineSize      0
dtype: int64


In [12]:
y_train = train.pop('price')
print(y_train)
# 검증 데이터인 price를 pop으로 y_train에 대입.

0       10298
1       10491
2       13498
3       10600
4       11000
        ...  
3754    10495
3755    17991
3756     6830
3757    11500
3758     8295
Name: price, Length: 3759, dtype: int64


## 수치형 활용

In [14]:
cols = ['year', 'mileage', 'tax', 'mpg', 'engineSize']
n_train = train[cols]
n_test = test[cols]
#수치형 데이터를 n_... 으로 대입

In [16]:
# 수치형만을 사용하여 회귀분석에 활용. 검증 데이터와 테스트 데이터 분리
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(n_train, y_train, test_size = 0.1, random_state = 2024) # 예측할 데이터 제외 / 예측할 데이터 위치
X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

((3383, 5), (376, 5), (3383,), (376,))

In [18]:
# 확률이 아닌 가격 분석이므로 회귀분석 진행. 랜덤포레스트
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)
print(pred)

[ 8224.37       16354.96       11178.12        9970.1
 13695.09        8831.83        8777.04       12472.77
 11034.16        9930.56       16037.26       11640.91
  6903.19       10222.49666667  4983.79       11778.06
  7740.17       17795.25       17363.35       16887.97
 12159.95       10399.64        7116.15       19697.42
 12337.49       10582.34       15799.18        4056.13
  8870.16        6367.16        8421.63       11437.23
  8519.85       10664.75       16074.17        9460.67
 12790.22       18194.95       11842.4        11478.09
 10587.28       19004.24       11670.36       20234.85
  6716.65       11612.38       11940.78533333 16742.21
  7927.97       15374.66       11798.96       10705.75
 18002.13        8854.57       11559.94       16034.28
  3846.3        22423.82       19734.04        9919.32
 18426.82        9605.21       10973.87        9755.62
  8494.42        9332.83        9523.38202381 14566.09
 10025.79       11283.82       17659.04        5565.56
  9501.56  

In [20]:
from sklearn.metrics import mean_squared_error
def rmse (a,b):
  return mean_squared_error(a, b) **0.5

In [21]:
rmse(y_val, pred)

1440.7019778832444

## 수치형 + 범주형 활용

In [22]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
y_train = train.pop("price")

In [23]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [24]:
train.shape, test.shape

((3759, 30), (1617, 30))

In [25]:
train.head(2)

,year,mileage,tax,mpg,engineSize,model_ B-MAX,model_ C-MAX,model_ EcoSport,model_ Edge,model_ Fiesta,...,model_ Puma,model_ S-MAX,model_ Tourneo Connect,model_ Tourneo Custom,transmission_Automatic,transmission_Manual,transmission_Semi-Auto,fuelType_Diesel,fuelType_Hybrid,fuelType_Petrol
0,2017,25013,150,53.3,1.0,False,False,True,False,False,...,False,False,False,False,False,True,False,False,False,True
1,2016,30970,0,74.3,1.5,False,False,False,False,False,...,False,False,False,False,False,True,False,True,False,False


In [26]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, y_train, test_size = 0.15, random_state = 2024)
X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

((3195, 30), (564, 30), (3195,), (564,))

In [32]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)
# print(pred)


from sklearn.metrics import mean_squared_error
def rmse(a,b):
  return mean_squared_error(a,b) ** 0.5

print(rmse(y_val, pred))

1197.3672417031478


## Test 예측

In [37]:
pred = rf.predict(test)
pred2 = pred.astype(int)
print(pred2)
result = pd.DataFrame({
    'pred':pred
})
result.to_csv("result.csv", index=False)


[15509 15601 14641 ...  9842 13726  4911]


In [35]:
pd.read_csv('result.csv')

,pred
0,15509.34
1,15601.04
2,14641.20
3,18590.58
4,6440.59
...,...
1612,10936.47
1613,18153.55
1614,9842.31
1615,13726.31


In [38]:
y = pd.read_csv("y.csv")
rmse(y, pred2)

1423.0130421468298

In [ ]:
pred2 = pred.astype(int)
rmse(y, pred2)

1433.9775649471646